In [1]:
import numpy as np
import scipy as sp
import suite2p as s2p
import TwoPUtils
import os
from matplotlib import pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [5]:
write_folder = os.path.join('/mnt','BigDisk','2P_scratch')


mouse = '4467975.2'

file_list = (#{'date': '09_10_2020', 'scene':'TwoTower_foraging','session':2,'scan':2},
             #{'date': '11_10_2020', 'scene':'TwoTower_foraging','session':2,'scan':3},
             #{'date': '13_10_2020', 'scene':'TwoTower_foraging','session':2,'scan':3},
             #{'date': '14_10_2020', 'scene':'TwoTower_foraging','session':2,'scan':6},
             {'date': '15_10_2020', 'scene':'TwoTower_foraging','session':3,'scan':10},
             #{'date': '16_10_2020', 'scene':'TwoTower_foraging','session':3,'scan':7},
             #{'date': '17_10_2020', 'scene':'TwoTower_foraging','session':2,'scan':3},
             #{'date': '17_10_2020', 'scene':'TwoTower_foraging','session':4,'scan':8}, # dendrites'
             #{'date': '17_10_2020', 'scene':'TwoTower_foraging','session':6,'scan':12}, # dendrites
             )

In [3]:
def check_and_make_h5s(file_list,runh5=True):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
        try:
            source_folder  = os.path.join('/media','mplitt','Backup Plus','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus1','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        h5path = os.path.join(file_stem,scan_str+".h5")
        if not os.path.exists(h5path) and runh5:
            _ = TwoPUtils.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=True)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [6]:
for f in file_list:
    _h5list,_frs = check_and_make_h5s([f,],runh5=False)
    ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'nchannels':2,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'h5py_key':'data',
                                    'nchannels':2,
                                    'threshold_scaling':2.,
                                    'chan2_thresh':.01})
    ops=s2p.run_s2p(ops=ops)
    !rm {_h5list[0]}/*.h5 
    

{'date': '15_10_2020', 'scene': 'TwoTower_foraging', 'session': 3, 'scan': 10}
['/mnt/BigDisk/2P_scratch/4467975.2/15_10_2020/TwoTower_foraging/TwoTower_foraging_003_010']
{}
FOUND BINARIES AND OPS IN ['/mnt/BigDisk/2P_scratch/4467975.2/15_10_2020/TwoTower_foraging/TwoTower_foraging_003_010/suite2p/plane0/ops.npy']
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
NOTE: applying default /home/mplitt/.suite2p/classifiers/classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 11
Binned movie [2070,416,694], 171.03 sec.
NOTE: estimated spatial scale ~6 pixels, time epochs 1.73, threshold 17.25 
0 ROIs, score=245.66
1000 ROIs, score=17.73
Found 1018 ROIs, 42.75 sec
After removing overlaps, 986 ROIs remain
Masks made in 13.49 sec.
>>>> CELLPOSE estimating masks in anatomical channel
ERROR importing or running cellpose, continuing without anatomical estimates
----------- Total 245.57 sec.
----------- EXTRACT